In [ ]:
# -*- coding: utf-8 -*-
"""
PPO Dynamic Pricing with Gymnasium + RPT Integrated
Tích hợp đầy đủ: RPT Demand → PPO Pricing → Giá đề xuất
Không phụ thuộc file rpt.py / news_embeding.py
"""

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pickle
import json
import pandas as pd
from datetime import datetime, timedelta
import os
import warnings
import gymnasium as gym
from gymnasium import spaces
import sys
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/AI_for_Business_Project')
warnings.filterwarnings("ignore")

# Config Set Up

In [ ]:
RPT_MODEL_PATH = "/content/drive/MyDrive/Colab_Notebooks/AI_for_Business_Project/Output/rpt_demand_best.pth"
PREPROCESSED_NPZ = "/content/drive/MyDrive/Colab_Notebooks/AI_for_Business_Project/Output/preprocessed_data.npz"
SCALERS_PKL = "/content/drive/MyDrive/Colab_Notebooks/AI_for_Business_Project/Output/scalers.pkl"
PPO_MODEL_SAVE = "/content/drive/MyDrive/Colab_Notebooks/AI_for_Business_Project/Output/ppo_pricing_best.pth"

## PPO hyperparameters

In [ ]:
STATE_DIM = 3 + 8 + 3 + 45 + 11  # demand + env + available + news + rev_hist(10) + current_rev
ACTION_DIM = 3
HIDDEN_DIM = 256
PPO_EPOCHS = 4
CLIP_EPS = 0.2
GAMMA = 0.99
LAMBDA = 0.95
BATCH_SIZE = 32
LEARNING_RATE = 3e-4
MAX_GRAD_NORM = 0.5
PARITY_PENALTY = 1e7  # Phạt cực nặng nếu vi phạm

## Price Constraints



In [ ]:
MIN_PRICE = {'single': 800000, 'double': 1200000, 'vip': 2500000}
MAX_PRICE = {'single': 3000000, 'double': 5000000, 'vip': 10000000}

PRICE_PARITY = {
    'double_single_ratio': (1.3, 1.8),
    'vip_single_ratio': (2.0, 3.0)
}

WINDOW_SIZE = 60  # Đã dùng đúng trong sliding window

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load scalers
with open(SCALERS_PKL, 'rb') as f:
    scalers = pickle.load(f)
price_scaler = scalers['price']

# Import RPT
import sys
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/AI_for_Business_Project')
from rpt import RPTModel, predict_demand

ModuleNotFoundError: No module named 'rpt'